In [36]:
import gradio as gr
from transformers import pipeline
import pandas as pd
from msclap import CLAP
import numpy as np

In [ ]:

pipe = pipeline(task="zero-shot-audio-classification", model="laion/clap-htsat-fused")

In [59]:
#empty array 
audio_array = np.empty([1,1])

word_detected = False

In [53]:
#CLAP model's prediction function
def classify(audio):
    samplerate, array = audio

    #append live audio to audio array
    audio_array.append(array)
        
    result = pipe(audio_array, candidate_labels=["quiet", "turn left","turn right","press reload"])

    #if word is detected or if the audio reaches 5 seconds, 
    #then reset the audio array to empty
    if (word_detected == True) or (audio_array.size >= (samplerate*5)):
        audio_array = np.empty([1,1])
        word_detected = False
    
       
    df = pd.DataFrame(result)
    return df

In [58]:
#Settings for Gradio Interface
demo = gr.Interface(
    fn=classify,
    inputs=gr.Audio(sources=["microphone"],streaming=True),
    outputs=[gr.Dataframe()],
    live=True,
    
)

In [57]:
#Launches Gradio Interface
demo.launch()

Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1687, in process_api
    inputs = self.preprocess_data(fn_index, inputs, state, explicit_call)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jacobviveros/anaconda3/envs/yako/lib/python3.11/site-packages/gradio/blocks.py", line 1421, in preprocess_data
    processed_input.append(block.preprocess(inputs_cached))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
clapmodel = CLAP( version="clapcap", use_cuda=False)

In [ ]:
def caption(audio):
    samplerate, array = audio
    captions = clapmodel.generate_caption(array)
    return captions["text"]

In [ ]:
capdisplay = gr.Interface(
    fn=caption,
    inputs=gr.Audio(sources=["microphone"],streaming=True),
    outputs=["text"],
    live=True,
    
).launch()